In [1]:
import pandas as pd

In [18]:
def get_all():
    base = pd.read_excel(r'/home/gustavo/Documentos/ListedOff/data/Base.xlsx', engine='openpyxl')
    return base

In [17]:
def handling_base(base):
    base['Instrument'] = base['Ticker'].apply(lambda x: " ".join(x.split()[:-1]))
    base['Instrument'] = [t[:-1] for t in base['Instrument']]
    base = x.groupby(['YellowKey','Instrument']).first().reset_index()
    base = base[['YellowKey', 'Instrument', 'Maturity Label', 'Style', 'Maturity', 'Underlying', 'Mx Instrument']]
    base.to_csv(r'/home/gustavo/Documentos/ListedOff/data/Base_aux.csv', index=False)

def read_base_aux():
    base = pd.read_csv(r'/home/gustavo/Documentos/ListedOff/data/Base_aux.csv')
    return base

In [4]:
def tickers_to_booking():
    tickers = pd.read_excel(r'/home/gustavo/Documentos/ListedOff/data/Tickers.xlsx', header=None, engine='openpyxl')
    tickers = tickers.rename(columns={0:'Ticker_Full'})
    return tickers

In [19]:
def get_ticker_yellow_key():
    df = tickers_to_booking()
    df['Ticker'] = df['Ticker_Full'].apply(lambda x: " ".join(x.split()[:-1]))
    df['YellowKey'] = [y.split()[-1] for y in df['Ticker_Full']]
    df['Instrument'] = df['Ticker'].apply(lambda x: " ".join(x.split()[:-1]))
    df['Instrument'] = [t[:-1] for t in df['Instrument']]
    return df

In [20]:
get_ticker_yellow_key()

,Ticker_Full,Ticker,YellowKey,Instrument
0,S J1C 3000 Index,S J1C 3000,Index,S J1
1,SCH1P 3200 Comdty,SCH1P 3200,Comdty,SCH1
2,SCH1C 3000 Comdty,SCH1C 3000,Comdty,SCH1
3,SCJ1C 3000 Comdty,SCJ1C 3000,Comdty,SCJ1


In [21]:
def check_ticker():
    base = get_all()
    tickers = get_ticker_yellow_key()
    base_aux = read_base_aux()
    tickers['New'] = ''
    
    for t,i,j,y in zip(tickers.Ticker, tickers.Instrument, tickers.index, tickers.YellowKey):
        if base.Ticker.str.contains(t).any():
            df = base.loc[base['Ticker'] == t]
            if df.YellowKey.str.contains(y).any():
                tickers['New'][j] = 0
            else:
                pass
        elif base_aux.Instrument.str.contains(i).any():
            if base_aux.Instrument.str.contains(i).any():
                df = base_aux.loc[base_aux['Instrument'] == i]
                if df.YellowKey.str.contains(y).any():
                    tickers['New'][j] = 1
                else:
                    tickers['New'][j] = 2
            else:
                tickers['New'][j] = 2
        else:
            tickers['New'][j] = 2
    
    return tickers      

In [22]:
x = check_ticker()
x

,Ticker_Full,Ticker,YellowKey,Instrument,New
0,S J1C 3000 Index,S J1C 3000,Index,S J1,1
1,SCH1P 3200 Comdty,SCH1P 3200,Comdty,SCH1,1
2,SCH1C 3000 Comdty,SCH1C 3000,Comdty,SCH1,0
3,SCJ1C 3000 Comdty,SCJ1C 3000,Comdty,SCJ1,2


In [8]:
def construct_dataframe(base_aux, instrument, yellow_key, ticker):
    df = pd.DataFrame()
    df['Ticker'] = ''
    if base_aux.Instrument.str.contains(instrument):
        df = base_aux.loc[base_aux['Instrument'] == instrument]
        if df.YellowKey.str.contains(yellow_key):
             df['Ticker'] = ticker
    return df
    
    

In [36]:
def get_payout_strike():
    tickers = check_ticker()
    strike = [t.split()[-1]for t in tickers.Ticker]
    payout =  list(tickers['Ticker'].apply(lambda x: " ".join(x.split()[:-1])))
    payout = [p[-1] for p in payout]
    payout = ["Call" if p == "C" else "Put" for p in payout]
    tickers['Strike'] = strike
    tickers['Payout'] = payout
    
    return tickers

In [38]:
def merge_dfs():
    base_aux = read_base_aux()
    tickers = get_payout_strike()
    df_to_upload = pd.DataFrame()
    for t,y,i in zip(tickers.Ticker, tickers.YellowKey, tickers.Instrument):
        df = pd.DataFrame(data=[t,y,i], columns=['Ticker','YellowKey', 'Instrument'])
        df_aux = base_aux.loc[(base_aux['Instrument'] == i) & (base_aux['YellowKey'] == y)]
        df_to_upload = df_to_upload.append(pd.merge(df, 
                                     df_aux, 
                                     on='Instrument', 
                                     how='left'))
    return df_to_upload

In [40]:
merge_dfs()

ValueError: Shape of passed values is (3, 1), indices imply (3, 3)

In [37]:
get_payout_strike()

,Ticker_Full,Ticker,YellowKey,Instrument,New,Strike,Payout
0,S J1C 3000 Index,S J1C 3000,Index,S J1,1,3000,Call
1,SCH1P 3200 Comdty,SCH1P 3200,Comdty,SCH1,1,3200,Put
2,SCH1C 3000 Comdty,SCH1C 3000,Comdty,SCH1,0,3000,Call
3,SCJ1C 3000 Comdty,SCJ1C 3000,Comdty,SCJ1,2,3000,Call


In [28]:
x,y = check_ticker()
for i in x:
    print(i)
y

Ticker SCH1C 3000 já consta na base
O ticker SCJ1C 3000 é novo


,Ticker_Full,Ticker,YellowKey,Instrument,New
0,S J1C 3000 Index,S J1C 3000,Index,S J1,False
1,SCH1P 3200 Comdty,SCH1P 3200,Comdty,SCH1,False
3,SCJ1C 3000 Comdty,SCJ1C 3000,Comdty,SCJ1,True
